# APNet-official
[![Generic badge](https://img.shields.io/badge/GitHub-apnet-9cf.svg)][github]
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)][notebook]

Author: [tarepan]

[github]:https://github.com/tarepan/APNet-official
[notebook]:https://colab.research.google.com/github/tarepan/APNet-official/blob/main/apnet.ipynb
[tarepan]:https://github.com/tarepan

## Colab Check
Check environments.

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}' # Google Colaboratory runnning time
!head -n 1 /proc/driver/nvidia/gpus/**/information                  # GPU type
!/usr/local/cuda/bin/nvcc --version | sed '4!d'                     # CUDA version
!python --version                                                   # Python version
!pip show torch | sed '2!d'                                         # PyTorch version

0.000576505days (49.81sec)
Model: 		 Unknown
Cuda compilation tools, release 11.8, V11.8.89
Python 3.10.11
Version: 2.0.1+cu118


## Setup
Install the package from user_name/repo_name public repository

In [ ]:

# GoogleDrive
from google.colab import drive
drive.mount('/content/gdrive')

# Dedicated dependencies install
# !pip install "torch==2.0.0" -q            # Based on your PyTorch environment
# !pip install "torchaudio==2.0.1" -q       # Based on your PyTorch environment
# !pip install 'lightning>=2.0.0,<2.1.0' -q # Based on your PyTorch environment
# !pip install git+https://github.com/tarepan/lightlightning.git
!pip install git+https://github.com/tarepan/speechcorpusy.git
!pip install git+https://github.com/tarepan/extorch.git

# repository install
# !pip uninstall packname -y -q
# !pip install git+https://github.com/user_name/repo_name -q
!git clone https://github.com/tarepan/APNet-official
%cd APNet-official

## Data

### Resampling

In [ ]:
import numpy as np
import torch
import torchaudio
import librosa
from speechcorpusy import load_preset
import soundfile as sf


sr_target = 16000


corpus = load_preset("JVS", root="/content/gdrive/MyDrive/ML_data", download=False)
corpus.get_contents()
all_uttr_paths = list(map(lambda id: corpus.get_item_path(id), corpus.get_identities()))

# Resampler
_, sr_source = librosa.load(all_uttr_paths[0], sr=None, mono=True)
resampler = torchaudio.transforms.Resample(sr_source, sr_target, resampling_method = 'sinc_interp_kaiser').cuda()

# Run
for p in all_uttr_paths:
    audio = librosa.load(p, sr=None, mono=True)[0].astype(np.float32)
    audio_resampled = resampler(torch.from_numpy(audio).cuda()).cpu().numpy()
    sf.write(p.with_suffix(".16k.wav"), audio_resampled, sr_target)


## Train

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/gdrive/MyDrive/ML_results/apnet-official

!python train.py